# Jovian: configure and start analysis

___
## Preparation
___

In [ ]:
%load test.txt

Add all raw Illumina PE data to a folder named "raw_data" relative from the location of this notebook.

Add samplesheet script of Mark, list it below

Add pipeline config file, list it below. Add a link to the edit page?

In [ ]:
# %load profile/pipeline_parameters.yaml
######################################
### General parameters             ###
######################################
###### GOTCHA WARNING
### Only use spaces in this file, NO tabs!

threads: # It's advisible efficiciency-wise to set these numbers only on a log2 scale (e.g. 2,4,8,16)
    Clean_the_data: 2
    HuGo_removal_pt1_alignment: 4
    HuGo_removal_pt2_Sam2Bam: 4
    HuGo_removal_pt4_extract_paired_unmapped_reads: 4
    HuGo_removal_pt5_extract_unpaired_unmapped_reads: 4
    De_novo_assembly: 12
    Taxonomic_classification_of_scaffolds: 12   # BLAST accepts a maximum of 16 threads, any value  >16 results in error
    Generate_contigs_metrics_pt1_alignment_of_reads_to_scaffolds: 4
    Generate_contigs_metrics_pt2_Sam2Bam: 4
    SNP_calling_pt1_calling: 12
    Fragment_length_analysis_pt2_alignment: 4
    quantify_output: 8 # For reading lengths of fastq files in parallel; WARNING: this is a local rule!

databases:                                                                                                               # Location of NT, NR and TaxDB are stored via an alias in your home directory (~/.ncbirc). They should work with keywords "nt" "nr" "taxdb"
    HuGo_ref: /mnt/db/Reference_genomes/Homo_sapiens/NCBI/GRCh38/Sequence/Bowtie2Index_without_EBV_virus_chr/genome.fa   # Location of HuGo assembly GRCH38 from which I manually removed the EBV chromosome, see the README.md for explanation  
    Krona_taxonomy: /mnt/db/taxonomy_krona/                                                                              # Location of the Krona taxonomy files, these are updated weekly via Crontab (thanks to Robert)
    virusHostDB: /mnt/db/Virus-Host_interaction_DB/virushostdb.tsv                                                       # Location of the Virus-Host interaction database, see their publication at .... TODO
    NCBI_new_taxdump_rankedlineage: /mnt/db/new_taxdump/rankedlineage.dmp.delim                                          # Location of the NCBI's "new_taxdump" file "rankedlineage.dmp", see their website at ...... TODO
    NCBI_new_taxdump_host: /mnt/db/new_taxdump/host.dmp.delim                                                            # Location of the NCBI's "new_taxdump" file "host.dmp", see their website at ....... TODO

typingtool_http:   #TEMP, remove this comment later: http://mpftypl01-ext-p.rivm.nl:8080 --> internal link (bypasses the RIVM proxy and thus the DDOS protection)
    NoV_TT_http: https://www.rivm.nl/mpf/typingservice/norovirus          # Link to the NoV typingtool (RIVM mirror), the webversion is accessible via https://www.rivm.nl/mpf/typingtool/norovirus
    EV_TT_http: https://www.rivm.nl/mpf/typingservice/enterovirus         # Link to the EV typingtool (RIVM mirror), the webversion is accessible via https://www.rivm.nl/mpf/typingtool/enterovirus
    test3_TT_http: https://www.rivm.nl/mpf/typingservice/test3            # Link to the "test3" typingtool (RIVM mirror), the webversion is accessible via https://www.rivm.nl/mpf/typingtool/test3

server_info:
    http_adress: http://rivm-biohn-l01p.rivm.ssc-campus.nl
    port: 8080

HTML_index_titles:
    IGVjs_title: "Interactive genome viewers for all samples in this run:"
    heatmap_title: "Interactive heatmaps of different taxonomic levels for all samples in this run:"

######################################
### Software parameters            ###
######################################
###### GOTCHA WARNING
### Only use spaces in this file, NO tabs!

Trimmomatic:                                                                                                    # http://www.usadellab.org/cms/?page=trimmomatic
    adapter_removal_config: ILLUMINACLIP:files/trimmomatic_0.36_adapters_lists/NexteraPE-PE.fa:2:30:10:8:true   # For the Nextera PE lib prep adapters
    quality_trimming_config: SLIDINGWINDOW:5:30                                                                 # Default: 5 nucleotides window size, minimum average Phred score of 30
    minimum_length_config: MINLEN:50                                                                            # Default: Remove anything smaller than 50 nucleotides
    
HuGo_removal:
    alignment_type: --local  # Choose if you want to do "local" or "global" alignment. Keep this as "--local" unless you know what you're doing.

SPAdes:                             # TODO: Latest version of SPAdes does this automically, need to update to new version first. See http://cab.spbu.ru/files/release3.12.0/manual.html section "Multi-cell data set with read lengths 2 x 250"
    kmersizes: 21,33,55,77         # uncomment this line if you want to use the pipeline for short Illumina reads (<250 nt in length). Then alos comment the line below.
#    kmersizes: 21,33,55,77,99,127   # uncomment this line if you want to use the pipeline for longer Illumina reads (>250 nt in length). Then also comment the line above.

scaffold_minLen_filter:
    minlen: 500             # Minimum allowed scaffold size to be allowed for downstream processessing. Advice, use a minimum length that is atleast 1nt greater than your Illumina read length

taxonomic_classification:   # NT database is hardcoded for now
    evalue: 0.05            # E-value threshold for saving hits
    max_target_seqs: 10     # Maximum number of target sequences to report

taxonomic_classification_LCA:
    bitscoreDeltaLCA: 5   # Every taxonomic hit within this bitscore distance will be used in the LCA analysis.

SNP_calling:
    max_cov: 20000     # See comments in the Snakefile rule "SNP_calling_pt1_calling" for explanation. Don't change this value unless you know what you're doing.
    minimum_AF: 0.05   # This is the minimum allelle frequency for which you want a SNP to be reported. Default is 5%.

ORF_prediction:
    procedure: meta      # Set the prodigal procedure. Use "meta" for metagenomics data.
    output_format: gff   # Set the ORF prediction output format. The format that IGVjs (the alignment viewer program) accepts is "gff"


Add snakemake configuration and cluster configuration file below.

In [2]:
%%bash
echo -e "________________________________________________________________________________\n\tContents of the config.yaml (contains the Snakemake CLI parameters):\n________________________________________________________________________________"
cat profile/config.yaml

________________________________________________________________________________
	Contents of the config.yaml (contains the Snakemake CLI parameters):
________________________________________________________________________________
### Profiles parameters: (See: https://snakemake.readthedocs.io/en/stable/executable.html?highlight=profile#profiles AND https://github.com/snakemake-profiles/doc )
#dryrun: false
#verbose: false
#printshellcmds: true
#quiet: false
#reason: true
#keep-going: false
notemp: true
cores: 120
resources:
    TT_connections=3
latency-wait: 60
local-cores: 4
use-conda: true
drmaa: " -q bio-prio -n {threads} -R \"span[hosts=1]\""
drmaa-log-dir: logs/drmaa
jobname: PZN_{name}.jobid{jobid}


Add link to demultiplexing notebook.

Embed tree HTML of all the log files

Add the snakemake start code to the cell below. NB, quiet mode.

Link to the report notebook

Add Jovian logo

______________
______________
______________

#### N.B. This can probably be automated later

First move to the parentfolder where you want to do your analysis.

In [ ]:
cd /data     # Replace the "/data" with the path where you want to go

Then, pull the latest code from Gitlab via:

In [ ]:
# Pseudocode, still need to check
git clone http://rivm-git/PZN.git

Rename the newly created folder to the name of your analysis.

In [ ]:
# Pseudocode, still need to check
mv PZN_name [my_analysis_name]
mkdir [my_analysis_name]/raw_data

Now, move the files you want to analyse to your analysis folder and in the subfolder named "raw_data". N.B. they need to be paired Illumina fastq's in order to work properly. Use an FTP-program for this, e.g. Filezilla.

___
## Work in progress, pseudocode
___

##### N.B. The parts below are doable, but will take a lot of time to implement. I've postponed it for now

Edit your sample sheet:

This would allow specifying file locations anywhere on the filesystem, and is required to accurately work with metadata file.

In [3]:
# Placeholder for jupyter editor of sample sheet
# Layout:
#    Sample_name    Sample_path_R1                                 Sample_path_R1
#    sample_1       /data/location/to/my/data/sample_1_R1.fastq    /data/location/to/my/data/sample_1_R2.fastq
#    sample_2       /data/location/to/my/data/sample_2_R1.fastq    /data/location/to/my/data/sample_2_R2.fastq
#    sample_etc     /data/location/to/my/data/sample_etc_R1.fastq  /data/location/to/my/data/sample_etc_R2.fastq

Edit your metatdata sheet:

This would allow sample-swap heuristics, different procedures based on metadata (e.g. different trimming for PE or SE data), and provides more clear questions and logging --> tracability.

In [ ]:
# Placeholder for jupyter editor of metadata sheet
# Layout:
#    Sample_name Matrix       Host     Gender    Seq_and_lib    Goal
#    sample_1    serum        Human    Female    Illumina PE    metagenomics
#    sample_2    feces        Human    Male      Illumina SE    metagenomics
#    sample_etc  cell-cult    Chimp    NA        Nanopore       Quasispecies

___
## Start your analysis
___

Run the cell below to see if there are any missing files or errors in the Snakefile. This is called a "dry-run".

In [ ]:
source activate PZN
snakemake -npj20 -s Snakefile

If there are no errors in the dry-run, perform the actual analysis via the code in the cell below.

In [ ]:
source activate PZN
snakemake -pj20 -s Snakefile

___
# Work in progress
___

Please note, the pipeline expects that the input files are stored in a folder named "raw_data". Also, it expects files are unpacked and are named as [sample_name]_R[1|2].fastq. If this is not the case, perform these commands first:

In [ ]:
# Unzip the files in folder raw_data using "parallel" to use full compute power

cd raw_data/
parallel gunzip ::: *.gz

In [ ]:
# To rename the files to an accepted format

# First by eye determine what is listed between the "_R[1|2]" and ".fastq". I.e. "_0001". Then insert that string into the code below.

for file in raw_data/*.fastq
do
    mv $file ${file/_0001/}
done

To do:
- Fix bug in bin/determine_GC

In [ ]:
RAW_DATA = "raw_data_test/"          # Here you set the input folder containing the input PE fastq's

#from os.path import join
#(SAMPLES, DELME,) = glob_wildcards(join(RAW_DATA, "{sample,[^/]+}{delme,_R[1|2]}.fastq"))

import glob
from os.path import join
FILES= glob.glob(join(RAW_DATA, "*.fastq")) # Glob all files (irrespective of R1 or R2)
FILES_R1= glob.glob(join(RAW_DATA, "*R1*.fastq")) # Glob all R1 files
FILES_R2= glob.glob(join(RAW_DATA, "*R2*.fastq")) # Glob all R2 files
SAMPLES = [i.split("/")[-1].split("_R1")[0] for i in files_R1] # Create sample basenames
SAMPLES_R1 = [i.split("/")[-1].split("_R1")[0] + "_R1" for i in files_R1] # Create sample basenames
SAMPLES_R2 = [i.split("/")[-1].split("_R2")[0] + "_R2" for i in files_R2] # Create sample basenames


print(FILES)
print(FILES_R1)
print(FILES_R2)
print(SAMPLES)
print(SAMPLES_R1)
print(SAMPLES_R2)

#samples_list = [i.split("_cov")[0] for i in samples_list_cov]
#basename_sample = sample.split("/")[-1]   # Remove the path from sample, thus, leave only the basename (extension has been removed priorly)


In [15]:
import os

samples = set([ f.split("_R")[0] for f in os.listdir("v2/raw_data") if f.endswith(".fastq")])
print(samples)

{'7_20688_AGGCAGAAGCGTAAGA_L001', '9_20689_TCCTGAGCGCGTAAGA_L001'}
